In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import urllib.request

nltk.download('stopwords')

# 1. Download dataset from UCI (SMS Spam Collection)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
filename = "smsspamcollection.zip"

# Download and unzip
import zipfile
import os

if not os.path.exists('SMSSpamCollection'):
    urllib.request.urlretrieve(url, filename)
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

# 2. Load dataset into DataFrame
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])

# 3. Preprocess text
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    words = text.split()
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)

df['message_clean'] = df['message'].apply(preprocess)

# 4. Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['message_clean'])
y = df['label'].map({'ham': 0, 'spam': 1})

# 5. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 6. Train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# 7. Evaluate model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.9730861244019139
Classification Report:
               precision    recall  f1-score   support

         Ham       0.97      1.00      0.98      1448
        Spam       1.00      0.80      0.89       224

    accuracy                           0.97      1672
   macro avg       0.98      0.90      0.94      1672
weighted avg       0.97      0.97      0.97      1672

Confusion Matrix:
 [[1448    0]
 [  45  179]]
